In [9]:
import pathlib
import pickle
import sys

import marqo

In [2]:
import numpy as np

In [3]:
sys.path.insert(0, "src")
import data_functions
import embedding_functions
import cluster_functions
import graph_functions
import util_functions
from marqo_external_utils import MarqoEmbeddingStore, MarqoDocumentStore, MarqoDocument

C:\Users\fra3066mat\PycharmProjects\concept-graphs\src\data_functions.py:15: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [70]:
from spacy.tokens import Doc

if not Doc.has_extension("doc_id"):
    Doc.set_extension("doc_id", default=None)
if not Doc.has_extension("doc_index"):
    Doc.set_extension("doc_index", default=None)
if not Doc.has_extension("doc_name"):
    Doc.set_extension("doc_name", default=None)
if not Doc.has_extension("doc_topic"):
    Doc.set_extension("doc_topic", default=None)
if not Doc.has_extension("offset_in_doc"):
    Doc.set_extension("offset_in_doc", default=None)
if not Doc.has_extension("negex"):
    Doc.set_extension("negex", default=None)

base_name_test = "grascco_lokal_test"
path_name_test = lambda x: f"./tmp/{base_name_test}/{base_name_test}_{x}.pickle"---

In [5]:
base_name = "grascco_lokal"
path_name = lambda x: f"./tmp/{base_name}/{base_name}_{x}.pickle"

In [5]:
base_name_test = "grascco_lokal_test"
path_name_test = lambda x: f"./tmp/{base_name_test}/{base_name_test}_{x}.pickle"

In [6]:
grascco_data_test = data_functions.DataProcessingFactory.load(
    data_obj_path=path_name_test("data"),
)

In [7]:
grascco_embedding_test = embedding_functions.SentenceEmbeddingsFactory.create(
    data_obj=grascco_data_test,
    cache_path=pathlib.Path(path_name_test("embedding")).parent,
    cache_name="_".join(pathlib.Path(path_name_test("embedding")).stem.split("_")[:-1]),
    model_name="Sahajtomar/German-semantic",
    storage_method=("vector_store", {},),
    vectorstore_normalizeEmbeddings=False,
    vectorstore_annParameters={
        "spaceType": "dotproduct",  #prenormalized-angular(default)
        "parameters": {
            "efConstruction": 1024,
            "m": 16
        }
    }
)

INFO:root:Creating Sentence Embedding with 'None'
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: Sahajtomar/German-semantic
Batches: 100%|██████████| 28/28 [00:12<00:00,  2.25it/s]
2025-05-23 09:06:40,803 logger:'marqo' WARNING Your Marqo Python client requires a minimum Marqo version of 2.18.0 to function properly, but your Marqo version is 2.16.1. Please upgrade your Marqo instance to avoid potential errors. If you have already changed your Marqo instance but still get this warning, please restart your Python interpreter.
2025-05-23 09:06:43,531 logger:'marqo' INFO     add_documents batch 0: took 2.488s for Marqo to process & index 128 docs. Roundtrip time: 2.671s.
INFO:marqo:    add_documents batch 0: took 2.488s for Marqo to process & index 128 docs. Roundtrip time: 2.671s.
2025-05-23 09:06:44,402 logger:'marqo' INFO     add_documents batch 1: took 0.683s for Marqo to pr

Saved under: C:\Users\fra3066mat\PycharmProjects\concept-graphs\tmp\grascco_lokal_test\grascco_lokal_test.pickle


In [8]:
isinstance(grascco_data, data_functions.DataProcessingFactory.DataProcessing)

True

In [7]:
grascco_data = data_functions.DataProcessingFactory.load(
    data_obj_path=path_name("data"),
)
grascco_embedding = embedding_functions.SentenceEmbeddingsFactory.load(
    embeddings_obj_path=path_name("embedding"),
    data_obj=grascco_data,
)
grascco_clustering = cluster_functions.PhraseClusterFactory.load(
    cluster_obj_path=path_name("clustering"),
    embedding_obj=grascco_embedding,
)

In [ ]:
grascco_embedding_from_vec = embedding_functions.SentenceEmbeddingsFactory.load(
    data_obj_path=path_name("data"),
    embeddings_obj_path=path_name("vec"),
    storage_method=("vector_store", {},),
)

In [8]:
grascco_graph_test = pickle.load(
    open(path_name_test("graph"), 'rb')
)

In [42]:
grascco_data = data_functions.DataProcessingFactory.load(
    data_obj_path=path_name("data"),
)

In [79]:
doc_0 = grascco_data.processed_docs[0]
print(doc_0.get_extension("negex"))
werte = [ch for ch in doc_0.noun_chunks][0]
werte_underscore = getattr(werte, "_")

(None, None, None, None)


In [81]:
getattr(doc_0, "_").__dict__

{'_extensions': {'doc_id': (None, None, None, None),
  'doc_index': (None, None, None, None),
  'doc_name': (None, None, None, None),
  'doc_topic': (None, None, None, None),
  'offset_in_doc': (None, None, None, None),
  'negex': (None, None, None, None)},
 '_obj': Werte Frau Kollegin, werter geehrter Herr Kollege!,
 '_doc': Werte Frau Kollegin, werter geehrter Herr Kollege!,
 '_start': None,
 '_end': None}

In [68]:
hasattr(werte, "_") and getattr(getattr(werte, "_"), "negex", False)

False

---

In [10]:
embedded_phrases = [
    (chunk["text"], grascco_embedding.sentence_embeddings[idx]) for idx, chunk in
    enumerate(grascco_embedding.data_processing_obj.data_chunk_sets)
    if 8 in [d["id"] for d in chunk["doc"]]
]

In [11]:
phrases, embeddings = zip(*embedded_phrases)
test_doc = MarqoDocument(
    phrases=phrases,
    embeddings=embeddings,
)

In [24]:
test_doc.as_tuples

[('werte',
  array([ 0.3701871 ,  0.31832993,  0.08824714, ...,  0.09354529,
          0.56552315, -0.23733714], dtype=float32)),
 ('frau kollegin',
  array([-0.1903553 ,  0.17692809,  0.29991037, ..., -0.7145708 ,
         -0.16699101, -0.5400057 ], dtype=float32)),
 ('patientin',
  array([ 0.2600834 ,  0.48468646, -0.18760607, ...,  0.09251131,
         -0.42281944, -0.15168679], dtype=float32)),
 ('aufnahme',
  array([ 0.8548429 , -0.5273871 ,  0.3986764 , ...,  0.13365792,
          0.19517979, -0.66357213], dtype=float32)),
 ('verlauf',
  array([ 0.12985386, -0.20498803,  0.57968867, ...,  0.6519341 ,
          1.1352999 , -1.024749  ], dtype=float32)),
 ('mg',
  array([ 0.23868962,  0.15230763, -0.25467727, ...,  0.13008146,
         -0.44494236,  0.49546698], dtype=float32)),
 ('therapie',
  array([-0.25470355,  0.63473725, -0.21538383, ..., -0.34303823,
          0.15202348,  0.990997  ], dtype=float32)),
 ('procedere',
  array([ 1.2050004 ,  0.31515324,  0.6631018 , ...,  0.34

---

In [10]:
mq = marqo.Client("http://localhost:8882")

In [13]:
mq.index("grascco_smaller_footprint_embedding").delete()

{'acknowledged': True}

In [12]:
mqs = MarqoEmbeddingStore("http://localhost:8882", "grascco_lokal_test")
mqds = MarqoDocumentStore(embedding_store=mqs)

In [15]:
mqs.marqo_index.recommend(["10"], exclude_input_documents=False)

{'hits': [{'graph_cluster': ['2'],
   '_id': '10',
   'phrase': 'handamputation',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 491.0687255859375},
  {'graph_cluster': ['2'],
   '_id': '238',
   'phrase': 'handchirurgen',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 359.72576904296875},
  {'_id': '51',
   'phrase': 'abgetrennte hand',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 345.06219482421875},
  {'_id': '55',
   'phrase': 'komplette amputation',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 344.34820556640625},
  {'graph_cluster': ['2'],
   '_id': '20',
   'phrase': 'handreplantaticm',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 337.96893310546875},
  {'graph_cluster': ['2'],
   '_id': '229',
   'phrase': 'handchirurgischer sicht',
   '_highlights': [{'phrase_vector': ''}],
   '_score': 313.2591247558594},
  {'graph_cluster': ['2'],
   '_id': '647',
   'phrase': 'handgelenksfraktur',
   '_highlights': [{'phrase_vector': ''}],


In [31]:
mqs.marqo_index.get_document("877")

{'graph_cluster': ['1'], '_id': '877', 'phrase': 'universitätsklinik'}

In [33]:
mqs.store_size

959

In [41]:
mqs.marqo_index.get_document("886")

{'graph_cluster': ['4'], '_id': '886', 'phrase': 'stationären behandlung'}

In [35]:
[(i, mqs.best_hits_for_field(str(i), delete_if_not_similar=False)) for i in range(877, mqs.store_size)]

[(877, [('1', 11.760240059120495)]),
 (878, [('1', 15.563912360431399)]),
 (879, [('1', 13.69795101801703), ('4', 3.972856571368427)]),
 (880, []),
 (881, []),
 (882, []),
 (883, [('1', 5.941574448092673)]),
 (884, []),
 (885, []),
 (886, [('4', 13.746252881286466), ('1', 1.9939149636221862)]),
 (887, []),
 (888, []),
 (889, []),
 (890, []),
 (891, [('4', 17.582121253832234)]),
 (892, []),
 (893, []),
 (894, [('4', 1.9940438019040638)]),
 (895, []),
 (896, []),
 (897, []),
 (898, []),
 (899, []),
 (900, []),
 (901, []),
 (902, []),
 (903, []),
 (904, []),
 (905, []),
 (906, []),
 (907, []),
 (908, []),
 (909, []),
 (910, []),
 (911, []),
 (912, []),
 (913, []),
 (914, []),
 (915, []),
 (916, [('2', 1.9919543034489404)]),
 (917, [('2', 9.847546933007932)]),
 (918, []),
 (919, []),
 (920, [('1', 7.902381302323839)]),
 (921, []),
 (922, []),
 (923, []),
 (924, []),
 (925, []),
 (926, []),
 (927, []),
 (928, [('4', 5.931212460122179)]),
 (929, []),
 (930, [('0', 3.9662730218590942)]),
 (93

In [14]:
for i, g in enumerate(grascco_graph_test):
    doc_list = []
    for n, d in g.nodes(True):
        doc_list.append(
            {
                "_id": str(n),
                "graph_cluster": [str(i)]
            }
        )
    _res = mqs.marqo_index.update_documents(
        documents=doc_list,
        # client_batch_size=128,
        # use_existing_tensors=True
    )

In [18]:
mqds.add_document(test_doc)

2025-05-23 09:08:02,890 logger:'marqo' INFO     add_documents batch 0: took 0.625s for Marqo to process & index 116 docs. Roundtrip time: 0.789s.
INFO:marqo:    add_documents batch 0: took 0.625s for Marqo to process & index 116 docs. Roundtrip time: 0.789s.
INFO:root:For id '877' the same embedding is already in the index with id '0'.
INFO:root:For id '878' the same embedding is already in the index with id '1'.
INFO:root:For id '879' the same embedding is already in the index with id '49'.
INFO:root:For id '880' the same embedding is already in the index with id '54'.
INFO:root:For id '881' the same embedding is already in the index with id '65'.
INFO:root:For id '882' the same embedding is already in the index with id '152'.
INFO:root:For id '883' the same embedding is already in the index with id '190'.
INFO:root:For id '884' the same embedding is already in the index with id '210'.
INFO:root:For id '885' the same embedding is already in the index with id '211'.
INFO:root:For id '8

---

In [ ]:
grascco_embedding.data_processing_obj.data_chunk_sets[1]

In [ ]:
vector_name = "phrase_vector"


def create_index_document(sentence_embedding_obj: 'embedding_functions.SentenceEmbeddingsFactory.SentenceEmbeddings'):
    for i, embedding in enumerate(sentence_embedding_obj.sentence_embeddings):
        _chunk_obj = sentence_embedding_obj.data_processing_obj.data_chunk_sets[i]
        yield {
            "_id": str(i),
            "documents": [str(d['id']) for d in _chunk_obj['doc']],
            #"phrase": _chunk_obj['text'],
            vector_name: {
                "content": _chunk_obj['text'],
                "vector": [float(f) for f in embedding]
            }
        }

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

In [ ]:
index_name = "grascco_index"
settings = {
    "treatUrlsAndPointersAsImages": False,
    "model": "no_model",
    "modelProperties": {
        "dimensions": 1024,
        "type": "no_model",
    },
    "annParameters": {
        "spaceType": "dotproduct",  #prenormalized-angular(default)
        "parameters": {
            "efConstruction": 1024,
            "m": 16
        }
    }
}
try:
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )
except:
    mq.delete_index(index_name)
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )

In [ ]:
mq.index(index_name).add_documents(
    list(create_index_document(grascco_embedding)),
    client_batch_size=128,
    tensor_fields=[vector_name],
    mappings={
        vector_name: {
            "type": "custom_vector"
        }
    },
)

In [ ]:
mq.index("grascco_lokal_test").get_document("877", expose_facets=True)

In [ ]:
mqs.store_embedding(("handamputation",
                     mq.index("grascco_lokal_test").get_document("10", expose_facets=True)["_tensor_facets"][0][
                         "_embedding"]))

In [ ]:
mqs.best_hits_for_field("10")

In [ ]:
mqs.marqo_index.delete_documents(["877"])

In [ ]:
mq.index("grascco_lokal_test").get_documents(["10", "11"])

In [ ]:
mq.index("grascco_lokal_test").recommend(
    documents=['877'],
    tensor_fields=["phrase_vector"],
    limit=2,
    exclude_input_documents=False
).get("hits")

In [ ]:
ex_doc = mq.index("grascco_lokal_test").get_document("10", expose_facets=True)
ex_doc

In [ ]:
mq.index("grascco_lokal_test").search(
    q={
        "customVector": {
            "vector": ex_doc["_tensor_facets"][0]["_embedding"],
            "content": ex_doc["phrase"],
        }
    },
    search_method="HYBRID"
)

In [ ]:
_ids = [str(i) for i in range(6)]
np.asarray([
    _res["_tensor_facets"][0]["_embedding"]
    for _res in
    mq.index(index_name).get_documents(_ids, expose_facets=True)["results"]
]).tolist()

In [ ]:
mq.index(index_name).get_stats()["numberOfDocuments"]

In [ ]:
mq.index(index_name).get_document("5180")

In [ ]:
marqo_store = MarqoEmbeddingExternal(
    client_url="http://localhost:8882",
    index_name="grascco_lokal_test"
)

In [ ]:
mq.index("grascco_lokal_test").search(
    q='psychose',
    search_method="LEXICAL",
)

---

In [1]:
from marqo import Client

mq = Client()
index_name = "iron-docs"

# Optionally delete the index if it already exists
try:
    mq.index(index_name).delete()
except:
    pass

# Create the index with custom settings
index_settings = {
    "model": "e5-base-v2",
    "normalizeEmbeddings": True,
    "textPreprocessing": {
        "splitLength": 3,
        "splitOverlap": 1,
        "splitMethod": "sentence"
    },
}
mq.create_index(index_name, settings_dict=index_settings)

2025-08-20 15:20:50,969 logger:'marqo' WARNING Your Marqo Python client requires a minimum Marqo version of 2.18.0 to function properly, but your Marqo version is 2.16.1. Please upgrade your Marqo instance to avoid potential errors. If you have already changed your Marqo instance but still get this warning, please restart your Python interpreter.


{'acknowledged': True, 'index': 'iron-docs'}